$\LARGE{African\hspace{0.1cm}Institute\hspace{0.1cm}For\hspace{0.1cm}Mathematical\hspace{0.1cm}Sciences,\hspace{0.1cm}AIMS-Sénégal\\}$

$\Large{Cours\hspace{0.1cm}de: Deep\hspace{0.1cm}Learning\hspace{0.1cm}and\hspace{0.1cm}Neural\hspace{0.1cm}Network\\}$

Responsable du cours: $\textbf{Prof. Stephane KOUAMO}$

$\large{\textbf{Projet portant sur la prédiction de l'activité agricole au cameroon en appliquant  une structure de réseau de neurone.}}$

Projet présenté par: $\textbf{Mikhaël Presley KIBINDA-MOUKENGUE}$

In [963]:
import pandas as pd
import numpy as np
import time
from keras.layers import Dropout, Dense, Activation
from keras.models import Sequential

In [964]:
from keras.utils import to_categorical

## Affichage du dataset

In [965]:
df=pd.read_csv('don_RN.csv')

In [966]:
df.head()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent
0,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,1.0,2.0,NaN,NaN,4.0
1,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,2.0,2.0,NaN,NaN,4.0
2,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,3.0,2.0,NaN,NaN,4.0
3,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,4.0,1.0,2.0,1.0,4.0
4,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,5.0,1.0,1.0,1.0,4.0


### Vérification des données manquantes

In [967]:
df.isna().sum()

Region                   0
Depart                   0
Arond                    0
Ville                    0
Village_quart            0
Longitude                0
Latitude                 0
Result_collect         240
Milieu_Red               0
Code_spe                 0
Prodution_pecul          1
Dest_principal      201331
implicaion_femme    201332
code_agent               0
dtype: int64

#### Vérification de la ligne de "Prodution_pecul" qui a une valeur manquante

In [968]:
loc=df[df[['Prodution_pecul']].isnull().any(axis=1)]

In [969]:
loc

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent
97314,4.0,21.0,139.0,2.0,2.0,14.9396,12.09216,8.0,2.0,10.0,NaN,NaN,NaN,55.0


#### Suppréssion de cette ligne

In [970]:
df.drop(97314,0,inplace=True)

In [971]:
df.head()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent
0,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,1.0,2.0,NaN,NaN,4.0
1,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,2.0,2.0,NaN,NaN,4.0
2,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,3.0,2.0,NaN,NaN,4.0
3,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,4.0,1.0,2.0,1.0,4.0
4,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,5.0,1.0,1.0,1.0,4.0


### Copie de notre dataset

In [972]:
dff=df.copy

## Statistique des données

#### Taille ou dimension du dataset

In [973]:
df.shape

(260019, 14)

In [974]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Region,260019.0,4.988832,2.567918,1.0,3.00000,4.00000,7.00000,10.0
Depart,260019.0,27.062153,14.472168,1.0,18.00000,24.00000,38.00000,58.0
Arond,260019.0,173.994289,93.818561,1.0,110.00000,163.00000,246.00000,360.0
Ville,260019.0,3.443818,8.964635,0.0,2.00000,2.00000,4.00000,999.0
Village_quart,260019.0,23.640130,36.077804,0.0,5.00000,12.00000,27.00000,714.0
Longitude,260019.0,10.086845,6.992207,0.0,5.47731,10.53494,13.55092,522.0
Latitude,260019.0,9.582849,6.579964,0.0,5.60172,10.13752,12.01432,522.0
Result_collect,259779.0,1.002610,0.133851,1.0,1.00000,1.00000,1.00000,8.0
Milieu_Red,260019.0,1.828205,0.377203,1.0,2.00000,2.00000,2.00000,2.0
Code_spe,260019.0,7.999896,4.320490,1.0,4.00000,8.00000,12.00000,15.0


In [975]:
df.isna().sum() #Vérification à nouveau, des valeurs manquantes

Region                   0
Depart                   0
Arond                    0
Ville                    0
Village_quart            0
Longitude                0
Latitude                 0
Result_collect         240
Milieu_Red               0
Code_spe                 0
Prodution_pecul          0
Dest_principal      201330
implicaion_femme    201331
code_agent               0
dtype: int64

## Suppréssion des colonnes ['Dest_principal','implicaion_femme']

In [976]:
data=df.drop(columns=['Dest_principal','implicaion_femme'])
data.isna().sum()  # Vérification des valeurs manquantes

Region               0
Depart               0
Arond                0
Ville                0
Village_quart        0
Longitude            0
Latitude             0
Result_collect     240
Milieu_Red           0
Code_spe             0
Prodution_pecul      0
code_agent           0
dtype: int64

## Affichage du dataset dont la colonne 'Result_collect' contient uniquement les valeurs manquantes

In [977]:
X_1=data[data['Result_collect'].isnull()]

In [978]:
X_1

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,code_agent
27525,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,1.0,2.0,11.0
27526,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,2.0,2.0,11.0
27527,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,3.0,2.0,11.0
27528,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,4.0,2.0,11.0
27529,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,5.0,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
257615,10.0,58.0,352.0,3.0,3.0,9.04432,4.290573,NaN,2.0,11.0,2.0,73.0
257616,10.0,58.0,352.0,3.0,3.0,9.04432,4.290573,NaN,2.0,12.0,2.0,73.0
257617,10.0,58.0,352.0,3.0,3.0,9.04432,4.290573,NaN,2.0,13.0,2.0,73.0
257618,10.0,58.0,352.0,3.0,3.0,9.04432,4.290573,NaN,2.0,14.0,2.0,73.0


# First model

## Affichage du dataset ayant aucune valeur manquante

In [979]:
data_1=data.dropna(subset=['Result_collect'])

#### Dimension

In [980]:
data_1.shape

(259779, 12)

#### Vérification des valeurs manquantes

In [981]:
data_1.isna().sum()

Region             0
Depart             0
Arond              0
Ville              0
Village_quart      0
Longitude          0
Latitude           0
Result_collect     0
Milieu_Red         0
Code_spe           0
Prodution_pecul    0
code_agent         0
dtype: int64

# Echantillonage, Séparation en Features et Target

### Features

In [1030]:
X=data_1.drop(columns=['Result_collect']).astype('int32')

### Target

In [1031]:
y=data1[['Result_collect']].astype('int32')

# Splitting de notre échantillon en train et test

In [1032]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

### Dimensions des training et testing sets

In [1033]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((181845, 11), (77934, 11), (181845, 1), (77934, 1))

# Structure du Réseau de Neurone

In [1037]:
model = Sequential()
model.add(Dense(29, input_dim=11, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='softmax'))

In [1038]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Entrainement du modèle

In [1039]:
start_time = time.time()
model.fit(X_train, y_train, epochs=5, batch_size=256, verbose = 1)
temps_exec = time.time() - start_time
print("\nTemps passé à entraîner le modèle pour 5 epochs : %.3f secondes" % temps_exec)

Epoch 1/5
181845/181845 [==============================] - 1s 5us/step - loss: -0.0351 - accuracy: 0.9997
Epoch 2/5
181845/181845 [==============================] - 1s 5us/step - loss: -0.0351 - accuracy: 0.9997
Epoch 3/5
181845/181845 [==============================] - 1s 5us/step - loss: -0.0351 - accuracy: 0.9997
Epoch 4/5
181845/181845 [==============================] - 1s 5us/step - loss: -0.0351 - accuracy: 0.9997
Epoch 5/5
181845/181845 [==============================] - 1s 5us/step - loss: -0.0351 - accuracy: 0.9997

Temps passé à entraîner le modèle pour 5 epochs : 5.049 secondes


### Prédiction

In [1040]:
y_pred=pd.DataFrame(model.predict(X_test))

## Vérification de la performance du modèle

In [1041]:
scores = model.evaluate(X_test, y_test, batch_size = 256, verbose = 1)
print("Le taux de réussite est de : %.2f%%" % (scores[1] * 100))

77934/77934 [==============================] - 0s 3us/step
Le taux de réussite est de : 99.95%


# Second model

### Remplacement des valeurs manquantes de la colonne 'Result_collect' par ses valeurs predictes

In [1042]:
df[['Result_collect']].fillna(y_pred.mode())

,Result_collect
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
260015,1.0
260016,1.0
260017,1.0
260018,1.0


### Suppréssion de la colonne 'Result_collect' dans le dataset 'df' du depart

In [1043]:
df_11=df.drop(columns=['Result_collect'])

### Concaténation de la colonne 'Result_collect' sans valeurs manquantes avec 'df_11' puis création d'un nouveau dataset 'df_1'

In [1044]:
df_1=pd.concat([df_11, df[['Result_collect']].fillna(1)],axis = 1)

In [1045]:
df_1.head()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent,Result_collect
0,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,2.0,NaN,NaN,4.0,1.0
1,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,2.0,2.0,NaN,NaN,4.0,1.0
2,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,3.0,2.0,NaN,NaN,4.0,1.0
3,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,4.0,1.0,2.0,1.0,4.0,1.0
4,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,5.0,1.0,1.0,1.0,4.0,1.0


## Vérification des valeurs manquantes dans 'df_1'

In [1046]:
df_1.isnull().sum()

Region                   0
Depart                   0
Arond                    0
Ville                    0
Village_quart            0
Longitude                0
Latitude                 0
Milieu_Red               0
Code_spe                 0
Prodution_pecul          0
Dest_principal      201330
implicaion_femme    201331
code_agent               0
Result_collect           0
dtype: int64

### Dimension de 'df_1'

In [1047]:
df_1.shape

(260019, 14)

### Copie de 'df_1'

In [1048]:
df_f=df_1.copy

### Suppréssion de la colonne 'implicaion_femme'

In [1049]:
dataa=df_1.drop(columns=['implicaion_femme'])

## Affichage du dataset dont la colonne 'Dest_principal' contient uniquement les valeurs manquantes

In [1050]:
X_2=dataa[dataa['Dest_principal'].isnull()]

## Affichage du dataset ayant aucune valeur manquante

In [1051]:
data_2=dataa.dropna(subset=['Dest_principal'])

#### Dimension

In [1052]:
data_2.shape

(58689, 13)

#### Vérification des valeurs manquantes

In [1053]:
data_2.isna().sum()

Region             0
Depart             0
Arond              0
Ville              0
Village_quart      0
Longitude          0
Latitude           0
Milieu_Red         0
Code_spe           0
Prodution_pecul    0
Dest_principal     0
code_agent         0
Result_collect     0
dtype: int64

## Echantillonage, Séparation en Features et en target

### Features

In [1054]:
X=data_2.drop(columns=['Dest_principal']).astype('int32')

### Target

In [1055]:
y=data_2[['Dest_principal']].astype('int32')

## Splitting de l'echantillon, en train et test sets

In [1056]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

#### Dimension de nos training et testing sets

In [1057]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((41082, 12), (17607, 12), (41082, 1), (17607, 1))

## Structure du reseau de neurone

In [1063]:
model = Sequential()
model.add(Dense(29, input_dim=12, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='softmax'))

In [1064]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Entrainement du modèle

In [1065]:
start_time = time.time()
model.fit(X_train, y_train, epochs=5, batch_size=256, verbose = 1)
temps_exec = time.time() - start_time
print("\nTemps passé à entraîner le modèle pour 5 epochs : %.3f secondes" % temps_exec)

Epoch 1/5
41082/41082 [==============================] - 0s 7us/step - loss: -17.7365 - accuracy: 0.6061
Epoch 2/5
41082/41082 [==============================] - 0s 5us/step - loss: -17.7365 - accuracy: 0.6061
Epoch 3/5
41082/41082 [==============================] - 0s 5us/step - loss: -17.7365 - accuracy: 0.6061
Epoch 4/5
41082/41082 [==============================] - 0s 5us/step - loss: -17.7365 - accuracy: 0.6061
Epoch 5/5
41082/41082 [==============================] - 0s 5us/step - loss: -17.7365 - accuracy: 0.6061

Temps passé à entraîner le modèle pour 5 epochs : 1.531 secondes


### Prédiction

In [1066]:
y_pred=pd.DataFrame(model.predict(X_test))

## Performance du modèle

In [1067]:
scores = model.evaluate(X_test, y_test, batch_size = 256, verbose = 1)
print("Le taux de réussite est de : %.2f%%" % (scores[1] * 100))

17607/17607 [==============================] - 0s 4us/step
Le taux de réussite est de : 60.71%


# Third model

### Remplacement des valeurs manquantes de la colonne 'Dest_principal' par ses valeurs predictes

In [1068]:
y_pred.mode()

,0
0,1.0


In [1069]:
df_1[['Dest_principal']].fillna(1)

,Dest_principal
0,1.0
1,1.0
2,1.0
3,2.0
4,1.0
...,...
260015,1.0
260016,1.0
260017,1.0
260018,4.0


### Suppréssion de la colonne 'Dest_principal' dans le dataset 'df_1'

In [1070]:
df_111=df_1.drop(columns=['Dest_principal'])

### Concaténation de la colonne 'Dest_principal' sans valeurs manquantes avec 'df_111' puis création d'un nouveau dataset 'df_2'

In [1071]:
df_2=pd.concat([df_111, df_1[['Dest_principal']].fillna(1)],axis = 1)

In [1072]:
df_2.head()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Milieu_Red,Code_spe,Prodution_pecul,implicaion_femme,code_agent,Result_collect,Dest_principal
0,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,2.0,NaN,4.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,2.0,2.0,NaN,4.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,3.0,2.0,NaN,4.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,4.0,1.0,1.0,4.0,1.0,2.0
4,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,5.0,1.0,1.0,4.0,1.0,1.0


### Vérification des valeurs manquantes dans 'df2'

In [1073]:
df_2.isnull().sum()

Region                   0
Depart                   0
Arond                    0
Ville                    0
Village_quart            0
Longitude                0
Latitude                 0
Milieu_Red               0
Code_spe                 0
Prodution_pecul          0
implicaion_femme    201331
code_agent               0
Result_collect           0
Dest_principal           0
dtype: int64

### Copie de df_2

In [1074]:
df_ff=df_2.copy

## Affichage du dataset dont la colonne 'implicaion_femme' contient uniquement les valeurs manquantes

In [1075]:
X_3=df_2[df_2['implicaion_femme'].isnull()]

## Affichage du dataset ayant aucune valeur manquante

In [1076]:
data_3=df_2.dropna(subset=['implicaion_femme']) 

## Echantillonage, Séparation en Features et en target

### Features

In [1077]:
X=data_3.drop(columns=['implicaion_femme']).astype('int32')

### Target

In [1078]:
y=data_3[['implicaion_femme']].astype('int32')

## Splitting de l'echantillon, en train et test sets

In [1079]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

#### Dimension de nos training et testing sets

In [1080]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((41081, 13), (17607, 13), (41081, 1), (17607, 1))

## Structure du réseau de neurone

In [1085]:
model = Sequential()
model.add(Dense(29, input_dim=13, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='softmax'))

In [1086]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Entrainement du modèle

In [1087]:
start_time = time.time()
model.fit(X_train, y_train, epochs=5, batch_size=256, verbose = 1)
temps_exec = time.time() - start_time
print("\nTemps passé à entraîner le modèle pour 5 epochs : %.3f secondes" % temps_exec)

Epoch 1/5
41081/41081 [==============================] - 0s 8us/step - loss: -2.1002 - accuracy: 0.4709
Epoch 2/5
41081/41081 [==============================] - 0s 5us/step - loss: -2.1002 - accuracy: 0.4709
Epoch 3/5
41081/41081 [==============================] - 0s 6us/step - loss: -2.1002 - accuracy: 0.4709
Epoch 4/5
41081/41081 [==============================] - 0s 6us/step - loss: -2.1002 - accuracy: 0.4709
Epoch 5/5
41081/41081 [==============================] - 0s 5us/step - loss: -2.1002 - accuracy: 0.4709

Temps passé à entraîner le modèle pour 5 epochs : 2.268 secondes


### Prédiction

In [1088]:
y_pred=pd.DataFrame(model.predict(X_test))

## Performance du modèle

In [1089]:
scores = model.evaluate(X_test, y_test, batch_size = 256, verbose = 1)
print("Le taux de réussite est de : %.2f%%" % (scores[1] * 100))

17607/17607 [==============================] - 0s 4us/step
Le taux de réussite est de : 46.81%


$$\Huge{\textbf{FIN}}$$